<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Build-image-for-simple-HTTP-test-app" data-toc-modified-id="Build-image-for-simple-HTTP-test-app-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Build image for simple HTTP test app</a></span></li><li><span><a href="#Run-containers-with-the-default-network-(&quot;bridge&quot;)" data-toc-modified-id="Run-containers-with-the-default-network-(&quot;bridge&quot;)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Run containers with the default network ("bridge")</a></span><ul class="toc-item"><li><span><a href="#Start-containers-and-get-their-IDs-and-mapped-ports" data-toc-modified-id="Start-containers-and-get-their-IDs-and-mapped-ports-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Start containers and get their IDs and mapped ports</a></span></li><li><span><a href="#Find-out-the-container-IPs-in-the-bridge-network" data-toc-modified-id="Find-out-the-container-IPs-in-the-bridge-network-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Find out the container IPs in the bridge network</a></span></li><li><span><a href="#Display-container-IPs,-mapped-ports,-and-container-IDs" data-toc-modified-id="Display-container-IPs,-mapped-ports,-and-container-IDs-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Display container IPs, mapped ports, and container IDs</a></span></li><li><span><a href="#Access-containers-with-bridge-network-IPs" data-toc-modified-id="Access-containers-with-bridge-network-IPs-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Access containers with bridge network IPs</a></span></li><li><span><a href="#Access-containers-with-mapped-host-ports" data-toc-modified-id="Access-containers-with-mapped-host-ports-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Access containers with mapped host ports</a></span></li><li><span><a href="#Containers-cannot-access-each-other-with-localhost" data-toc-modified-id="Containers-cannot-access-each-other-with-localhost-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Containers cannot access each other with <code>localhost</code></a></span></li><li><span><a href="#Cleanup" data-toc-modified-id="Cleanup-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Cleanup</a></span></li></ul></li><li><span><a href="#Use-the-host-network" data-toc-modified-id="Use-the-host-network-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Use the host network</a></span></li></ul></div>

# TODO
* add examples for '--network host' mode
* add examples for containers with shared custom network which has DNS

## Build image for simple HTTP test app

In [1]:
docker build -t simple_http_test_app simple_http_test_app

Sending build context to Docker daemon  5.632kB
Step 1/7 : FROM python:3
 ---> e32be9a6f71f
Step 2/7 : RUN pip3 install --upgrade pip
 ---> Using cache
 ---> 1c39c5ef7b40
Step 3/7 : COPY requirements.txt .
 ---> Using cache
 ---> ee1f9247ac3f
Step 4/7 : RUN pip3 install -r requirements.txt
 ---> Using cache
 ---> f6f7147d43eb
Step 5/7 : COPY app.py .
 ---> Using cache
 ---> b049e34952f0
Step 6/7 : EXPOSE 8080
 ---> Using cache
 ---> 34ec71d9d3b5
Step 7/7 : CMD ["python", "app.py"]
 ---> Using cache
 ---> f43d5045081f
Successfully built f43d5045081f
Successfully tagged simple_http_test_app:latest


## Run containers with the default network ("bridge")

### Start containers and get their IDs and mapped ports
Note that there are multiple ways to define port mappings:
* `-p 32001:8080` maps the container port 8080 to the host port 32001 (works only if 32001 is not used yet).
* `-p :8080` maps the container port 8080 to a free port on the host.
* `-P` maps all ports which are exposed in the Dockerfile to free ports on the host.

In [2]:
CONTAINER_ID_A=$(docker run -d --rm --name container-a -p :8080 -e 'NAME=app-a' simple_http_test_app)
CONTAINER_ID_B=$(docker run -d --rm --name container-b -p :8080 -e 'NAME=app-b' simple_http_test_app)

The mapped host ports can be obtained with `docker port`.

In [3]:
docker port $CONTAINER_ID_A

8080/tcp -> 0.0.0.0:32768


In [4]:
docker port $CONTAINER_ID_A 8080

0.0.0.0:32768


In [5]:
PORT_A=$(docker port $CONTAINER_ID_A 8080 | cut -d ':' -f 2)
PORT_B=$(docker port $CONTAINER_ID_B 8080 | cut -d ':' -f 2)

### Find out the container IPs in the bridge network

In [6]:
IP_A=$(docker inspect $CONTAINER_ID_A -f '{{ .NetworkSettings.IPAddress }}')
IP_B=$(docker inspect $CONTAINER_ID_B -f '{{ .NetworkSettings.IPAddress }}')

### Display container IPs, mapped ports, and container IDs

In [7]:
echo $IP_A $PORT_A $CONTAINER_ID_A
echo $IP_B $PORT_B $CONTAINER_ID_B

172.17.0.4 32768 34679fbca00315c559c9d064894bc7dc909a9d879b8c53b6c73da96265392008
172.17.0.5 32769 e75685c97769fcb1fbf5666216d220e69c50969b2278b01c9ee3330917aeb89d


### Access containers with bridge network IPs

In [8]:
curl $IP_A:8080/ping

response from app-a running on 34679fbca003 (172.17.0.4) on port 8080


In [9]:
curl $IP_B:8080/ping

response from app-b running on e75685c97769 (172.17.0.5) on port 8080


In [10]:
curl $IP_A:8080/access/http://$IP_B:8080/ping

forwarded by app-a running on 34679fbca003 (172.17.0.4) on port 8080:
  response from app-b running on e75685c97769 (172.17.0.5) on port 8080



### Access containers with mapped host ports

In [11]:
HOST_IP=$(hostname -I | cut -d ' ' -f 1)

In [12]:
curl $HOST_IP:$PORT_A/ping

response from app-a running on 34679fbca003 (172.17.0.4) on port 8080


In [13]:
curl $HOST_IP:$PORT_B/ping

response from app-b running on e75685c97769 (172.17.0.5) on port 8080


In [14]:
curl $HOST_IP:$PORT_A/access/http://$HOST_IP:$PORT_B/ping

forwarded by app-a running on 34679fbca003 (172.17.0.4) on port 8080:
  response from app-b running on e75685c97769 (172.17.0.5) on port 8080



### Containers cannot access each other with `localhost`
Note that only the host can access mapped container ports via `localhost`/127.0.0.1. Containers cannot access each other this way, they either have to use their IPs and the container ports, or the host IP and the mapped ports (both options are shown above).

In [15]:
curl 127.0.0.1:$PORT_A/ping

response from app-a running on 34679fbca003 (172.17.0.4) on port 8080


This does not work:

    curl 127.0.0.1:$PORT_A/access/http://127.0.0.1:$PORT_B/ping

### Cleanup

In [16]:
for ID in $CONTAINER_ID_A $CONTAINER_ID_B
do
    docker kill $ID
done

34679fbca00315c559c9d064894bc7dc909a9d879b8c53b6c73da96265392008
e75685c97769fcb1fbf5666216d220e69c50969b2278b01c9ee3330917aeb89d


## Use the host network
Containers can also be attached directly to the host network with `--network host`. No port mappings are needed then, and containers can access each other directly because they share the same network and IP.